In [77]:
from google.colab import drive
drive.mount('/content/drive')

DATA_PATH = "/content/drive/MyDrive/datathon/data/"

import pandas as pd

SEED = 42

frames = []
for year in range(2023, 2025):
    try:
        df = pd.read_csv(f"{DATA_PATH}{year}_LoL_esports_match_data_from_OraclesElixir.csv")
        frames.append(df)
    except FileNotFoundError:
        print(f"File not found for year {year}")
        continue

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-77-a783411a829c>:13: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{DATA_PATH}{year}_LoL_esports_match_data_from_OraclesElixir.csv")
<ipython-input-77-a783411a829c>:13: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{DATA_PATH}{year}_LoL_esports_match_data_from_OraclesElixir.csv")


In [78]:
# 전체(원본) 데이터셋
all_data = pd.concat(frames)

# 리그 데이터셋
# league_data = pd.concat(frames)
# league_data = league_data[league_data["league"].isin(["LCK", "MSI", "WLDs"])]


# 사용가능 데이터셋
complete_data = pd.concat(frames)
complete_data = complete_data[complete_data["datacompleteness"].isin(["complete"])]

all_data.shape,complete_data.shape

((243456, 161), (206784, 161))

In [79]:
# 열 삭제
complete_data = complete_data.drop(columns=['url','datacompleteness','monsterkillsownjungle','monsterkillsenemyjungle',
                                            'gspd', 'gpr', 'firstdragon', 'dragons', 'elementaldrakes', 'infernals',
                                            'mountains', 'clouds', 'oceans', 'chemtechs', 'hextechs', 'dragons (type unknown)',
                                            'elders', 'firstherald', 'heralds', 'void_grubs', 'opp_void_grubs', 'firstbaron',
                                            'firsttower', 'towers', 'firstmidtower', 'firsttothreetowers', 'turretplates', 'opp_turretplates'])
#20,25 들어간 열 삭제
columns_to_drop = [col for col in complete_data.columns if '20' in col or '25' in col]
complete_data = complete_data.drop(columns=columns_to_drop)

all_data.shape, complete_data.shape

((243456, 161), (206784, 103))

In [80]:
object_cols = ['opp_dragons', 'opp_elementaldrakes', 'opp_elders', 'opp_heralds', 'barons', 'opp_barons', 'opp_towers', 'inhibitors', 'opp_inhibitors']
# object_df = complete_data[object_cols] #데이터프레임으로 변경시
# 오브젝트는 0으로 결측치
for col in object_cols:
    complete_data[col] = complete_data[col].fillna(0)

complete_data[object_cols].isnull().sum().sum(),complete_data.shape

(0, (206784, 103))

In [81]:
banpick_cols = ['champion', 'ban1', 'ban2', 'ban3', 'ban4', 'ban5',
                'pick1', 'pick2', 'pick3', 'pick4', 'pick5']
# 챔피언 및 밴픽은 "UNK"으로 결측치 처리
for col in banpick_cols:
    complete_data[col] = complete_data[col].fillna("UNK")

complete_data[banpick_cols].isnull().sum().sum(),complete_data.shape

(0, (206784, 103))

In [82]:
kill_cols = ['kills', 'deaths', 'assists', 'teamkills', 'teamdeaths', 'doublekills', 'triplekills','quadrakills', 'pentakills',
             'damagetochampions', 'dpm', 'damageshare', 'damagetakenperminute', 'damagemitigatedperminute']

#킬과 데미지는 0으로 처리
for col in kill_cols:
    complete_data[col] = complete_data[col].fillna(0)

complete_data[kill_cols].isnull().sum().sum(),complete_data.shape

(0, (206784, 103))

In [83]:
# 스플릿 결측치
complete_data['split'] = complete_data['split'].fillna("UNK")
complete_data['split'].isnull().sum().sum(),complete_data.shape

(0, (206784, 103))

In [84]:
grouped = complete_data.groupby(['gameid', 'teamid'])

In [85]:
complete_data.shape

(206784, 103)

In [86]:
goldcs_cols = ['totalgold', 'earnedgold', 'earned gpm', 'earnedgoldshare', 'goldspent', 'total cs', 'minionkills',
               'monsterkills', 'cspm', 'goldat10', 'xpat10', 'csat10', 'opp_goldat10', 'opp_xpat10', 'opp_csat10',
               'golddiffat10', 'xpdiffat10', 'csdiffat10', 'killsat10', 'assistsat10', 'deathsat10', 'opp_killsat10',
               'opp_assistsat10', 'opp_deathsat10', 'goldat15', 'xpat15', 'csat15', 'opp_goldat15', 'opp_xpat15',
               'opp_csat15', 'golddiffat15', 'xpdiffat15', 'csdiffat15', 'killsat15', 'assistsat15', 'deathsat15',
               'opp_killsat15', 'opp_assistsat15', 'opp_deathsat15']

for col in goldcs_cols:
    if 'playerid' in complete_data.columns: #playerid있으면 goldcs_cols 결측치 0으로 채움
        complete_data[col] = complete_data[col].fillna(0)
    else:
        # gameid와 teamid로 팀별 평균 계산
        team_avg = complete_data.groupby(['gameid', 'teamid'])[col].transform('mean')
        # 결측치 평균으로 채우기
        complete_data[col] = complete_data[col].fillna(team_avg)
        # 평균계산후에도 결측치있으면 0으로 채우기
        complete_data[col] = complete_data[col].fillna(0)

complete_data[goldcs_cols].isnull().sum().sum(),complete_data.shape

(0, (206784, 103))

In [87]:
complete_data[goldcs_cols]

,totalgold,earnedgold,earned gpm,earnedgoldshare,goldspent,total cs,minionkills,monsterkills,cspm,goldat10,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
0,18855,13251,304.3874,0.295868,15955,399.0,367.0,32,9.1654,3163.0,...,131.0,322.0,263.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12082,6478,148.8055,0.144640,11675,159.0,23.0,136,3.6524,3035.0,...,117.0,-357.0,-1323.0,-43.0,0.0,0.0,0.0,0.0,0.0,0.0
2,15722,10118,232.4196,0.225914,14525,337.0,287.0,50,7.7412,3117.0,...,162.0,-479.0,-324.0,-26.0,0.0,0.0,0.0,0.0,0.0,0.0
3,17332,11728,269.4028,0.261862,15225,370.0,345.0,25,8.4992,3344.0,...,122.0,200.0,292.0,20.0,0.0,0.0,0.0,1.0,0.0,0.0
4,8816,3212,73.7825,0.071716,8125,21.0,17.0,4,0.4824,1953.0,...,3.0,-216.0,-579.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117547,16998,11529,271.6968,0.234899,15850,395.0,364.0,31,9.3087,2996.0,...,139.0,-759.0,283.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0
117548,19060,13591,320.2907,0.276913,17300,415.0,389.0,26,9.7800,3704.0,...,138.0,71.0,-114.0,-3.0,1.0,0.0,0.0,0.0,1.0,0.0
117549,10166,4697,110.6913,0.095694,8225,33.0,33.0,0,0.7777,2278.0,...,28.0,74.0,-185.0,-8.0,0.0,2.0,1.0,0.0,1.0,1.0
117550,75533,48186,1135.5695,0.000000,73333,0.0,1134.0,290,33.5585,15673.0,...,491.0,63.0,1625.0,46.0,3.0,4.0,4.0,4.0,4.0,3.0


In [88]:
ward_cols = ['wardsplaced', 'wpm', 'wardskilled', 'wcpm', 'controlwardsbought', 'visionscore', 'vspm']
#와드 평군값으로 결측치
for col in ward_cols:
    if 'playerid' in complete_data.columns: #playerid있으면 goldcs_cols 결측치 0으로 채움
        complete_data[col] = complete_data[col].fillna(0)
    else:
        # gameid와 teamid로 팀별 평균 계산
        team_avg = complete_data.groupby(['gameid', 'teamid'])[col].transform('mean')
        # 결측치 평균으로 채우기
        complete_data[col] = complete_data[col].fillna(team_avg)
        # 평균계산후에도 결측치있으면 0으로 채우기
        complete_data[col] = complete_data[col].fillna(0)

complete_data[ward_cols].isnull().sum().sum(),complete_data.shape

(0, (206784, 103))

In [89]:
complete_data.shape

(206784, 103)

In [90]:
list(complete_data.columns)

['gameid',
 'league',
 'year',
 'split',
 'playoffs',
 'date',
 'game',
 'patch',
 'participantid',
 'side',
 'position',
 'playername',
 'playerid',
 'teamname',
 'teamid',
 'champion',
 'ban1',
 'ban2',
 'ban3',
 'ban4',
 'ban5',
 'pick1',
 'pick2',
 'pick3',
 'pick4',
 'pick5',
 'gamelength',
 'result',
 'kills',
 'deaths',
 'assists',
 'teamkills',
 'teamdeaths',
 'doublekills',
 'triplekills',
 'quadrakills',
 'pentakills',
 'firstblood',
 'firstbloodkill',
 'firstbloodassist',
 'firstbloodvictim',
 'team kpm',
 'ckpm',
 'opp_dragons',
 'opp_elementaldrakes',
 'opp_elders',
 'opp_heralds',
 'barons',
 'opp_barons',
 'opp_towers',
 'inhibitors',
 'opp_inhibitors',
 'damagetochampions',
 'dpm',
 'damageshare',
 'damagetakenperminute',
 'damagemitigatedperminute',
 'wardsplaced',
 'wpm',
 'wardskilled',
 'wcpm',
 'controlwardsbought',
 'visionscore',
 'vspm',
 'totalgold',
 'earnedgold',
 'earned gpm',
 'earnedgoldshare',
 'goldspent',
 'total cs',
 'minionkills',
 'monsterkills',
 '

In [91]:
complete_data.to_csv(f"{DATA_PATH}train.csv",index=False)